In [2]:
import cv2
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import pandas as pd
import numpy
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import SGDRegressor
import pickle
import cv2


In [3]:
OLS_model = pickle.load(open('OLS.pkl', 'rb'))
SGD_model = pickle.load(open('SGD.pkl', 'rb'))
MeClf_model = pickle.load(open('Meclassifier.pkl', 'rb'))



In [4]:
img_dim = 64

In [9]:

cam = cv2.VideoCapture(0)

while True:
    check, frame = cam.read()

    cv2.imshow('video', frame)

    
    img_resize = cv2.resize(frame, (img_dim,img_dim))
    cur_image = img_resize.flatten()

    ols_pred = MeClf_model.predict([cur_image])
    if(ols_pred[0]):
        print("Unknown")
    else:
        print("Sameed")

    key = cv2.waitKey(1)
    if key == 27:
        break

cam.release()
cv2.destroyAllWindows()

Unknown
Unknown
Unknown
Unknown
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sameed
Sa

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


: 